In [ ]:
def calculate_scores(data):
    scores = {}

    # 1. Shares available to public and promoters' shares (5 points)
    shares_public = data.get('shares_public', 0)
    shares_pledged = data.get('shares_pledged', 0)
    if shares_pledged > 0:
        scores['shares'] = 2
    else:
      if shares_public > 15:
           scores['shares'] = 5
      else:
        scores['shares'] = 4

    # 2. Revenue growth YoY (10 points)
    revenue_growth = data.get('revenue_growth', 0) #5Y Double: 10/10, Inflation < 0
    scores['revenue_growth'] = min(max(revenue_growth, 0), 10)

    # 3. Net profit margin and YoY growth (10 points)
    profit_margin = data.get('profit_margin', 0)
    scores['profit_margin'] = min(max(profit_margin / 2, 0), 10)

    # 4. Market cap value (10 points)
    market_cap = data.get('market_cap', 0)
    assets = data.get('assets', 0)
    revenue = data.get('revenue', 0)
    liablities = data.get('liablities', 0)
    if market_cap > 15 * (assets + (profit_margin/100)*revenue - liablities):
        scores['market_cap'] = 0
    elif market_cap > 10 * (assets + (profit_margin/100)*revenue - liablities):
        scores['market_cap'] = 5
    else:
        scores['market_cap'] = 8

    # 5. Reverse DCF (15 points)
    reverse_dcf = data.get('reverse_dcf', 0)
    if reverse_dcf < 10:
      scores['reverse_dcf'] = 15
    elif reverse_dcf < 20:
      scores['reverse_dcf'] = 11
    elif reverse_dcf < 40:
      scores['reverse_dcf'] = 7
    else:
      scores['reverse_dcf'] = 3

    # 6. Free Cash Flow + Reserves (15 points)
    free_cash_flow = data.get('free_cash_flow', 0)
    reserves = data.get('reserves', 0)
    scores['cash_reserves'] = min(max((free_cash_flow + reserves) / 100, 0), 15)

    # 7. Get Debt to Assets ratio and previous debt (if available) from the data
    debt_to_equity = data.get('debt_to_equity', 0)
    previous_debt_to_equity = data.get('previous_debt_to_equity', 0)

    # Initialize the score to 0
    debt_score = 0

    #  Scoring based on the Debt to Assets ratio
    if debt_to_equity < 0.5:
      debt_score = 10  # Excellent financial health
    elif 0.5 <= debt_to_equity < 0.7:
      debt_score = 10 - (debt_to_equity - 0.5) * 20  # Penalize for higher debt
    else:
      debt_score = max(0, 10 - (debt_to_equity - 0.7) * 50)  # Strong penalty for high debt

    # Adjusting score based on the debt trend (increasing debt)
    if debt_to_equity > previous_debt_to_equity:
      debt_score = max(0, debt_score - 5)  # Penalty for increasing debt (deduct 2 points)

    # Final debt score
    scores['debt'] = debt_score


    # 8. Capital Work in Progress (5 points) Work in THIS!!!!
    cwip = data.get('cwip', 0)
    scores['cwip'] = 5 if cwip > 0 else 0

    # 9. Cash Conversion Cycle (10 points)
    cash_conversion_cycle = data.get('cash_conversion_cycle', 0)
    scores['cash_cycle'] = min(max(10 - cash_conversion_cycle / 10, 0), 10)

    # 10. Capex and ROCE (10 points)
    capex = data.get('capex', 0)
    revenue = data.get('revenue', 0)
    roce = data.get('roce', 0)

    # Calculate Capex/Revenue ratio
    capex_to_revenue = capex / revenue if revenue > 0 else 0

    # Scoring based on Capex/Revenue ratio and ROCE
    if capex_to_revenue < 0.05:
      scores['capex_roce'] = 3
    elif 0.05 <= capex_to_revenue < 0.1:
      scores['capex_roce'] = 8
    else:
      scores['capex_roce'] = 4

    # Adjust based on ROCE trends
    if roce > 15:
      scores['capex_roce'] += 2
    elif 10 <= roce <= 15:
      scores['capex_roce'] += 1
    else:
      scores['capex_roce'] += 0



    return scores

def main():

    data = {
        'shares_public': 33.76,
        'shares_pledged': 0,
        'revenue_growth': 31,  # Percent
        'profit_margin': 11.44,  # Percent
        'market_cap': 302,
        'assets': 58.36,
        'liablities': 24.76,
        'revenue': 72,
        'reverse_dcf': 60.80,
        'free_cash_flow': 0.72,
        'reserves': 13,
        'debt_to_equity': 0.26,
        'previous_debt_to_equity': 1.41,
        'cwip': 0,
        'cash_conversion_cycle': 66.13,
        'capex': 0.68,
        'roce': 27.75,
        }

    scores = calculate_scores(data)
    total_score = sum(scores.values())

    print("Detailed Scores:", scores)
    print("Total Score:", total_score, "/ 100")

if __name__ == "__main__":
    main()


Detailed Scores: {'shares': 5, 'revenue_growth': 10, 'profit_margin': 5.72, 'market_cap': 8, 'reverse_dcf': 3, 'cash_reserves': 0.13720000000000002, 'debt': 10, 'cwip': 0, 'cash_cycle': 3.3870000000000005, 'capex_roce': 5}
Total Score: 50.2442 / 100


In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_financial_data(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Dictionary to store extracted data
        financial_data = {
            'shares_public': None,
            'shares_pledged': None,
            'revenue_growth': None,
            'profit_margin': None,
            'market_cap': None,
            'assets': None,
            'liabilities': None,
            'revenue': None,
            'reverse_dcf': None,
            'free_cash_flow': None,
            'reserves': None,
            'debt_to_equity': None,
            'previous_debt_to_equity': None,
            'cwip': None,
            'cash_conversion_cycle': None,
            'capex': None,
            'roce': None,
        }

        # Example: Extract specific data (update selectors based on the page structure)
        # This is a placeholder; you'll need to inspect the page source to get accurate selectors.
        financial_data['shares_public'] = soup.select_one('div.shares-public').text.strip() if soup.select_one('div.shares-public') else None
        financial_data['shares_pledged'] = soup.select_one('div.shares-pledged').text.strip() if soup.select_one('div.shares-pledged') else None
        financial_data['revenue_growth'] = soup.select_one('div.revenue-growth').text.strip() if soup.select_one('div.revenue-growth') else None
        financial_data['profit_margin'] = soup.select_one('div.profit-margin').text.strip() if soup.select_one('div.profit-margin') else None
        # Repeat for other fields

        return financial_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
url = "https://www.tijorifinance.com/company/ipca-laboratories-limited/financials/"
data = scrape_financial_data(url)
if data:
    print(data)


In [ ]:
from bs4 import BeautifulSoup

# Simulated HTML snippet (replace with the HTML content of the page)
html = '''<https://www.tijorifinance.com/company/ipca-laboratories-limited/financials/>'''

# Parse the HTML
soup = BeautifulSoup(html, 'html.parser')

# Find the row containing "Assets"
assets_row = soup.find('span', text='Assets').find_parent('tr')

# Extract the numeric values from the 'td' tags in the same row
assets_data = [td.text.strip() for td in assets_row.find_all('td', class_='knowledge numericvalue text-right number')]

print("Assets Data:", assets_data)


<ipython-input-2-76965cb8858d>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  assets_row = soup.find('span', text='Assets').find_parent('tr')


AttributeError: 'NoneType' object has no attribute 'find_parent'